In [1]:
import sys
import cv2
import mediapipe as mp
import glob
import numpy as np
import json
import os
import time
from os.path import exists

In [2]:
datapath = '/mnt/HDD04/ASL_10_imit/outputs/webcam/cut/'
# vid_savepath = '/mnt/HDD04/ICASSP_2023/Data/annotated_videos/'
annot_path = datapath.replace('webcam/cut', 'annotated_images')
skelpath = annot_path.replace('annotated_images', 'skeletons')
frame_path = skelpath.replace('skeletons', 'images')

In [3]:
files = glob.glob(datapath + '*avi')
print('Num files present:', len(files))
print(files[0])

Num files present: 1924
/mnt/HDD04/ASL_10_imit/outputs/webcam/cut/2022_10_13_16_12_38_subjEmre_Exp0_class128_part4_word128_cnt4.avi


In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [5]:
f = files[0]
fname = f.split('/')[-1]
datapath + fname

'/mnt/HDD04/ASL_10_imit/outputs/webcam/cut/2022_10_13_16_12_38_subjEmre_Exp0_class128_part4_word128_cnt4.avi'

In [6]:
# extract skeleton
start_time = time.time()
cnt = 0
for f in files:
    fname = f.split('/')[-1]

    cnt += 1
    now = time.time()
    print('Processing file ' + str(cnt) + '/' + str(len(files)))
    cap = cv2.VideoCapture(datapath + fname)
    fps = cap.get(cv2.CAP_PROP_FPS)
    start_frame = 0
    end_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1
    subfolder_frame = frame_path + fname[:-4]
          
    if not exists(subfolder_frame):
        os.mkdir(subfolder_frame)
        # extract frames
#             for frame_no in range(start_frame, end_frame + 1):
        for frame_no in range(start_frame, end_frame + 1):
            cap.set(1,frame_no)  # Where frame_no is the frame you want
            ret, frame = cap.read()  # Read the frame
#                     frame = frame[bbox[1]:bbox[3], bbox[0]:bbox[2]]
            cv2.imwrite(subfolder_frame + '/' + fname[:-4] + '_frame' + f'{frame_no:05d}' + '.jpg', frame)

    # MediaPipe
    subfolder_skel = skelpath + fname.split('/')[-1][:-4]
    print(subfolder_skel)
    break
    if not exists(subfolder_skel):
        os.mkdir(subfolder_skel)
        IMAGE_FILES = sorted(glob.glob(subfolder_frame + '/*jpg'))
        with mp_hands.Hands(
            static_image_mode=True,
            max_num_hands=2,
            min_detection_confidence=0.5) as hands:

            image = cv2.flip(cv2.imread(IMAGE_FILES[0]), 1)
            fname2 = IMAGE_FILES[0].split('/')[-1]
            prev_results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            for idx, file in enumerate(IMAGE_FILES):
                # Read an image, flip it around y-axis for correct handedness output (see
                # above).
                image = cv2.flip(cv2.imread(file), 1)
                fname2 = file.split('/')[-1]
                # Convert the BGR image to RGB before processing.
                results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                # Print handedness and draw hand landmarks on the image.
#                     print('Frame: ', idx, '/', len(IMAGE_FILES))
                # print('Handedness:', results.multi_handedness)
                if not results.multi_hand_landmarks:
                    if not prev_results.multi_hand_landmarks:
                        continue
                    else:
                        results = prev_results  # to prevent frame elimination due to non-existent hands
                prev_results = results
                image_height, image_width, _ = image.shape
                annotated_image = image.copy()
                data = []

                for hand_id, hand_landmarks in enumerate(results.multi_hand_landmarks):


                  # print('hand_landmarks:', hand_landmarks)
                  # print(
                  #     f'Index finger tip coordinates: (',
                  #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
                  #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
                  # )
                  # print(dir(hand_landmarks.landmark[0]))
                  # print(hand_landmarks.landmark[0])
                  # print(hand_landmarks.landmark[0].x)

                    for land_id, mark in enumerate(hand_landmarks.landmark):

                        if land_id == 0:
                            x_temp = [mark.x]
                            y_temp = [mark.y]
                            z_temp = [mark.z]
                        else:
                            x_temp = np.concatenate((x_temp, [mark.x]), 0)
                            y_temp = np.concatenate((y_temp, [mark.y]), 0)
                            z_temp = np.concatenate((z_temp, [mark.z]), 0)

                    data.append(x_temp)
                    data.append(y_temp)
                    data.append(z_temp)
                    mp_drawing.draw_landmarks(
                          annotated_image,
                          hand_landmarks,
                          mp_hands.HAND_CONNECTIONS,
                          mp_drawing_styles.get_default_hand_landmarks_style(),
                          mp_drawing_styles.get_default_hand_connections_style())


                with open(subfolder_skel + '/' + fname2.replace('jpg', 'txt'), 'w') as f:
                    for elem in data:
                        for e in elem:
                            f.write(str(e) + ' ')
                        f.write('\n')
                subfolder_im = annot_path + fname[:-4]
                if not exists(subfolder_im):
                    os.mkdir(subfolder_im)
                cv2.imwrite(subfolder_im + '/' + fname2.replace('jpg', 'png'), cv2.flip(annotated_image, 1))

            # create video
#                     files = sorted(glob.glob(subfolder_im + '/*png'))
#                     frame = cv2.imread(files[0])
#                     height, width, layers = frame.shape
#                     video_name = vid_savepath + fname.split('/')[-1][:-4] + '.avi'
#                     video = cv2.VideoWriter(video_name, 0, fps, (width, height))
#                     for image in files:
#                         video.write(cv2.imread(image))
#                     video.release()

    print('Elapsed time:', time.time() - now, 'sec')
    print('Num. processed file:' + str(cnt) + '/' + str(len(files)))
    print('Total Elapsed time:', (time.time() - start_time)/60, ' min')

#             if j == 4:
#                 break
#             break

#     break


Processing file 1/1924
/mnt/HDD04/ASL_10_imit/outputs/skeletons/2022_10_13_16_12_38_subjEmre_Exp0_class128_part4_word128_cnt4
